In [ ]:
pip install Flask


In [ ]:
!pip -q install pyngrok

In [ ]:
pip install -r requirement.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 56.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of types-requests to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of types-requests to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
with open('/content/mlcb.md','r',encoding = 'utf-8') as file :
    document = file.read()


FileNotFoundError: [Errno 2] No such file or directory: '/content/mlcb.md'

In [ ]:
document  = document.casefold()

In [ ]:
import re
#document = 'source code cho chương này có thể được tìm thấy tại https://goo.gl/asf58q.\n\nmachine learning cơ bản https://machinelearningcoban.com\n---\n# chương 10\n\n# k-means clustering\n\n# 10.1 giới thiệu\n\nchúng ta đã làm quen với linear regression, một mô hình đơn giản trong supervised learning. trong chương này, một mô hình đơn giản khác trong unsupervised learning sẽ được trình bày. thuật toán này có tên là phân nhóm k-means (k-means clustering).\n\ntrong k-means clustering, chúng ta không biết nhãn của từng điểm dữ liệu'
a = len(document)

incorrect_title_pattern = r'---\n# chương (\d+). (.*?)\n\n'
# Loại bỏ các tiêu đề sai
document = re.sub(incorrect_title_pattern, '', document)
b = len(document)
print(a)
print(b)
c = len(incorrect_title_pattern)
print((a-b)/c)


In [ ]:
a = len(document)

redundant_str = r'\n\nmachine learning cơ bản\n\nhttps://machinelearningcoban.com\n'
# Loại bỏ các tiêu đề sai
document = re.sub(redundant_str, '', document)
b = len(document)
c = len(redundant_str)
print((a-b)/c)


In [ ]:
a = len(document)

redundant_str = r'\n\nmachine learning cơ bản https://machinelearningcoban.com\n'
# Loại bỏ các tiêu đề sai
document = re.sub(redundant_str, '', document)
b = len(document)
c = len(redundant_str)
print((a-b)/c)


In [ ]:
a = len(document)

redundant_str = r'machine learning cơ bản\n\nhttps://machinelearningcoban.com\n'
# Loại bỏ các tiêu đề sai
document = re.sub(redundant_str, '', document)
b = len(document)
c = len(redundant_str)
print((a-b)/c)


In [ ]:
a = len(document)

redundant_str = r'machine learning cơ bản\n\nhttps://machinelearningcoban.com'
# Loại bỏ các tiêu đề sai
document = re.sub(redundant_str, '', document)
b = len(document)
c = len(redundant_str)
print((a-b)/c)


In [ ]:
def insert_hash_this(long_string):
    """Inserts '#this' before each occurrence of the specified format."""
    pattern = r"(chương\s+\d+\n\n)"  # Capture the entire format
    replacement = r"#this\n\1"  # \1 refers to the captured group
    modified_string = re.sub(pattern, replacement, long_string, flags=re.MULTILINE)
    return modified_string
document = insert_hash_this(document)

In [ ]:
def split_at_hash_this(long_string):
    """Splits a string into a list at each occurrence of '#this'."""
    parts = re.split("chương\s+\d+\n\n" , long_string,flags = re.M) # Split at #this followed by newline
    return parts
document = split_at_hash_this(document)

In [ ]:
print(len(document))

In [ ]:

def get_title(text):
    lines = text.splitlines()
    return lines[0]
list_title = []
for doc in document :
    list_title.append(get_title(doc))


In [ ]:
list_title

In [ ]:
def extract_lines_starting_with_hash(long_string):
    return [re.sub(r"\d+", "", line[1:].strip()).strip()
            for line in re.findall(r"^#.*", long_string, re.MULTILINE)]
ut = []
for doc in document :
    ut.append(extract_lines_starting_with_hash(doc))

In [ ]:
  from semantic_router import Route
  from semantic_router.encoders import CohereEncoder, OpenAIEncoder
  from semantic_router.layer import RouteLayer

  # Định nghĩa các routes (giữ nguyên như trước)
  vu_huu_tiep = Route(
          name="vu_huu_tiep",
          utterances=ut[0],
      )
  loi_tac_gia =  Route(
          name="loi_tac_gia",
          utterances=ut[1][:17],
      )
  on_tap_dai_so_tuyen_tinh = Route(
          name="on_tap_dai_so_tuyen_tinh",
          utterances=ut[2],
      )
  giai_tich_ma_tran = Route(
          name="giai_tich_ma_tran",
          utterances=ut[3],
      )
  on_tap_xac_suat = Route(
          name="on_tap_xac_suat",
          utterances=ut[4],
      )
  maximum_likelihood_va_maximum_a_posteriori  = Route(
          name="maximum_likelihood_va_maximum_a_posteriori",
          utterances=ut[5],
      )
  cac_khai_niem_co_ban = Route(
          name="cac_khai_niem_co_ban",
          utterances=ut[6],
      )
  gioi_thieu_ve_feature_engineering = Route(
          name="gioi_thieu_ve_feature_engineering",
          utterances=ut[7] ,
      )
  linear_regression = Route(
          name="linear_regression",
          utterances=ut[8],
      )
  overfitting = Route(
          name="overfitting",
          utterances=ut[9],
      )
  k_nearest_neighbors = Route(
          name="k_nearest_neighbors",
          utterances=ut[10],
      )
  k_means_clustering = Route(
          name="k_means_clustering",
          utterances=ut[11],
      )
  naive_bayes_classifier = Route(
      name = 'naive_bayes_classifier' ,
      utterances = ut[12] ,
  )
  gradient_descent = Route(
          name="gradient_descent",
          utterances=ut[13],
      )
  perceptron_learning_algorithm = Route(
          name="perceptron_learning_algorithm",
          utterances=ut[14],
      )
  logistic_regression = Route(
          name="logistic_regression",
          utterances=ut[15],
      )
  softmax_regression = Route(
          name="softmax_regression",
          utterances=ut[16],
      )
  multilayer_neural_network_va_backpropagation = Route(
          name="multilayer_neural_network_va_backpropagation",
          utterances=ut[17],
      )
  content_based_recommendation_system = Route(
          name="content_based_recommendation_system",
          utterances=ut[18],
      )
  neighborhood_based_collaborative_filtering = Route(
          name="neighborhood_based_collaborative_filtering",
          utterances=ut[19],
      )
  matrix_factorization_collaborative_filtering = Route(
          name="matrix_factorization_collaborative_filtering",
          utterances=ut[20],
      )
  singular_value_decomposition = Route(
          name="singular_value_decomposition",
          utterances=ut[21],
      )
  principal_component_analysis = Route(
          name="principal_component_analysis",
          utterances=ut[22],
      )
  linear_discriminant_analysis = Route(
          name="linear_discriminant_analysis",
          utterances=ut[23],
      )
  tap_loi_va_ham_loi = Route(
          name="tap_loi_va_ham_loi",
          utterances=ut[24],
      )
  bai_toan_toi_uu_loi = Route(
          name="bai_toan_toi_uu_loi",
          utterances=ut[25],
      )
  duality = Route(
          name="duality",
          utterances=ut[26],
      )
  support_vector_machine = Route(
          name="support_vector_machine",
          utterances=ut[27],
      )
  soft_margin_support_vector_machine = Route(
          name="soft_margin_support_vector_machine",
          utterances=ut[28],
      )
  kernel_support_vector_machine = Route(
          name="kernel_support_vector_machine",
          utterances=ut[29],
      )
  multi_class_support_vector_machine = Route(
          name="multi_class_support_vector_machine",
          utterances=ut[30],
      )


  routes = [vu_huu_tiep,loi_tac_gia,on_tap_dai_so_tuyen_tinh,naive_bayes_classifier,giai_tich_ma_tran,on_tap_xac_suat,maximum_likelihood_va_maximum_a_posteriori,cac_khai_niem_co_ban,gioi_thieu_ve_feature_engineering,linear_regression,overfitting,k_nearest_neighbors,k_means_clustering,gradient_descent,perceptron_learning_algorithm,logistic_regression,softmax_regression,multilayer_neural_network_va_backpropagation,content_based_recommendation_system,neighborhood_based_collaborative_filtering,matrix_factorization_collaborative_filtering,singular_value_decomposition,principal_component_analysis,linear_discriminant_analysis,tap_loi_va_ham_loi,bai_toan_toi_uu_loi,duality,support_vector_machine,soft_margin_support_vector_machine,kernel_support_vector_machine,multi_class_support_vector_machine]

  os.environ["COHERE_API_KEY"] = ""
  encoder = CohereEncoder()

  # Tạo RouteLayer
  route_layer = RouteLayer(encoder=encoder, routes=routes,aggregation = 'max',)



In [ ]:
def get_topic_index_ifelse(topic):
    ind = -1  # Default value if topic is not found

    if topic == "vu_huu_tiep":
        ind = 0
    elif topic == "loi_tac_gia":
        ind = 1
    elif topic == "on_tap_dai_so_tuyen_tinh":
        ind = 2
    elif topic == "giai_tich_ma_tran":
        ind = 3
    elif topic == "on_tap_xac_suat":
        ind = 4
    elif topic == "maximum_likelihood_va_maximum_a_posteriori":
        ind = 5
    elif topic == "cac_khai_niem_co_ban":
        ind = 6
    elif topic == "gioi_thieu_ve_feature_engineering":
        ind = 7
    elif topic == "linear_regression":
        ind = 8
    elif topic == "overfitting":
        ind = 9
    elif topic == "k_nearest_neighbors":
        ind = 10
    elif topic == "k_means_clustering":
        ind = 11
    elif topic == 'naive_bayes_classifier' :
        ind = 12
    elif topic == "gradient_descent":
        ind = 13
    elif topic == "perceptron_learning_algorithm":
        ind = 14
    elif topic == "logistic_regression":
        ind = 15
    elif topic == "softmax_regression":
        ind = 16
    elif topic == "multilayer_neural_network_va_backpropagation":
        ind = 17
    elif topic == "content_based_recommendation_system":
        ind = 18
    elif topic == "neighborhood_based_collaborative_filtering":
        ind = 19
    elif topic == "matrix_factorization_collaborative_filtering":
        ind = 20
    elif topic == "singular_value_decomposition":
        ind = 21
    elif topic == "principal_component_analysis":
        ind = 22
    elif topic == "linear_discriminant_analysis":
        ind = 23
    elif topic == "tap_loi_va_ham_loi":
        ind = 24
    elif topic == "bai_toan_toi_uu_loi":
        ind = 25
    elif topic == "duality":
        ind = 26
    elif topic == "support_vector_machine":
        ind = 27
    elif topic == "soft_margin_support_vector_machine":
        ind = 28
    elif topic == "kernel_support_vector_machine":
        ind = 29
    elif topic == "multi_class_support_vector_machine":
        ind = 30

    return ind


In [ ]:
from langchain.docstore.document import Document
list_doc = []
for i in range(len(document)) :
    list_doc.append([Document(page_content = document[i])])

In [ ]:
text_splitter1 = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=100,separators = ['#','\n\n','\n'])
#text_splitter2 = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap=100,separators = ['#','\n\n','\n'])
list_splits = []
for i in range(len(document)) :
    # list_splits.append(text_splitter1.create_documents([document[i]]) + text_splitter2.create_documents([document[i]]))
    list_splits.append(text_splitter1.create_documents([document[i]]))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate

model_name = "hiieu/halong_embedding"  # Example model
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device' :"cuda"},
    encode_kwargs=encode_kwargs
)
list_retriever = []
# Embed
for i in range(len(list_splits)) :
    vectorstore = Chroma.from_documents(documents=list_splits[i],
                                    embedding=hf,persist_directory=f"chroma_db_{i}")
    print(vectorstore)
    list_retriever.append(vectorstore.as_retriever())




In [ ]:
# question = "null space là gì?"
# topic = route_layer(question).name
# ind = get_topic_index_ifelse(topic)
# print(ind)

In [ ]:
def generate(question) :
  topic = route_layer(question).name
  ind = get_topic_index_ifelse(topic)
  print(ind)
  template = """Hãy hiểu context bên dưới và hãy hiểu câu hỏi. Xem thông tin chỉ có trong context có thể dùng để trả lời một cách đầy đủ cho câu hỏi hay không.
  - Nếu không, thì trả lời: "Xin lỗi, tôi không có thông tin về câu hỏi này" và đưa ra 2 link trên internet có khả năng trả lời câu hỏi
  - Nếu có, thì hãy dùng nó để trả lời câu hỏi.
  Context: {context}
  Câu hỏi: {question}
  """
  prompt = ChatPromptTemplate.from_template(template)


  # LLM
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

  # Post-processing
  def format_docs(docs):
      return '<' + "\n".join(doc.page_content for doc in docs) + '>'

  # Chain
  rag_chain = (
      {"context": list_retriever[ind] | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )
  answer  = rag_chain.invoke(question)
  print(answer)
  return answer


In [ ]:
from flask import Flask, request, render_template
# Assuming your RAG chain is defined elsewhere and imported
from pyngrok import ngrok, conf

from google.colab import userdata
# Khởi tạo Flask app
app =  Flask(__name__)

# Định nghĩa tham số
IMAGE_PATH  = "static/output.jpg"
conf.get_default().auth_token = userdata.get('ngrok_token')

public_url = ngrok.connect(8888).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

@app.route("/", methods=["GET", "POST"])
def index():
    result = None
    if request.method == "POST":
        question = request.form.get("question")
        if question:
            try:
                result = generate(question)  # Pass as dictionary
                # Handle different output types from your RAG chain
                if isinstance(result, dict) and "answer" in result:
                    result = result["answer"] # Extract the answer if dictionary is returned
                elif isinstance(result, list) and len(result) > 0 and isinstance(result[0], dict) and "answer" in result[0]:
                    result = result[0]["answer"] # Extract the answer if list of dict is returned
                elif isinstance(result, str):
                    pass # Keep result as is if it is already a string
                else:
                    result = str(result) # Convert to string if it is other type
            except Exception as e:
                result = f"Error processing query: {e}"

    return render_template("index.html", result=result)

if __name__ =='__main__':
    app.run(debug=False, host='0.0.0.0', port=8888, use_reloader=False)